### Data Parsing

In [19]:
import os
import json 
import glob

def printConvo(userData, systemData):
    print("session id:", userData["session-id"])

    print(userData["task-information"]["goal"]["text"])

    temp = 0
    for speach in systemData["turns"]:
        print("system:", speach["output"]["transcript"])
        print("user:", userData["turns"][temp]["transcription"])
        temp += 1

def writeFiles(userData, systemData, conversationsFile, utterancesFile):
    
    conversationsFile.write("session id: %s\n" % userData["session-id"])
    conversationsFile.write("%s\n" % userData["task-information"]["goal"]["text"])

    temp = 0
    for speach in systemData["turns"]:
        
        dialogAct = userData["turns"][temp]["semantics"]["cam"]
        utteranceContent = userData["turns"][temp]["transcription"]
        
        conversationsFile.write("system: %s\n" %  speach["output"]["transcript"])
        conversationsFile.write("user: %s\n" % utteranceContent)
        utterancesFile.write("%s-%s\n" % (dialogAct.split("(")[0], utteranceContent.lower()))
        
        temp += 1
    
    conversationsFile.write("\n\n")

def writeFile(waiting):
    if waiting is False:
        conversationsFile = open("convo.txt", "w+")
        utterancesFile = open("utterances.txt", "w+")
    
    userFiles = glob.glob('**/label.json', recursive=True)
    systemFiles = glob.glob('**/log.json', recursive=True)
    i = 0
    for userFile in userFiles:
        with open(userFile) as f:
            userData = json.load(f)
        with open(systemFiles[i]) as f:
            systemData = json.load(f)
        i += 1
        if waiting is True :
            printConvo(userData, systemData)
            input('press Enter to display another chat both conversation')
        else: 
            writeFiles(userData, systemData, conversationsFile, utterancesFile)

    if waiting is False: 
        conversationsFile.close()
        utterancesFile.close()

writeFile(False)

### Splitting Sets

In [16]:
import numpy as np

utterancesFile = open("utterances.txt", "r")
utterancesData = {}
for line in utterancesFile:
    line = line.split("-")
    if line[0] not in utterancesData:
        utterancesData[line[0]] = []    
    utterancesData[line[0]].append(line[1].strip())
    
trainingData = {}
testData = {}
for dialogType in utterancesData:
    splitted = np.split(utterancesData[dialogType], [int(len(utterancesData[dialogType]) * 0.85)])
    trainingData[dialogType] = splitted[0].tolist()
    testData[dialogType] = splitted[1].tolist()

982
174
